In [15]:
!pip install datasets -q

In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!wget -c https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-11-29.tar.gz -O cord19.tar.gz

--2025-04-10 16:23:00--  https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-11-29.tar.gz
Resolving ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)... 52.92.186.178, 52.92.133.234, 3.5.77.152, ...
Connecting to ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)|52.92.186.178|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [18]:
!mkdir cord19
!tar -xvzf cord19.tar.gz -C cord19

mkdir: cannot create directory ‘cord19’: File exists
2020-11-29/changelog
2020-11-29/cord_19_embeddings.tar.gz
2020-11-29/document_parses.tar.gz
2020-11-29/metadata.csv


In [23]:
!tar -xzf /content/cord19/2020-11-29/document_parses.tar.gz -C /content/cord19/2020-11-29/

In [24]:
import os

fulltext_dir = "/content/cord19/2020-11-29/document_parses/pdf_json"
print(os.listdir(fulltext_dir)[:5])  # should now show some JSON files

['407dfb371c1bb8508e94ada9bea7cf86e55399b3.json', '7208e9f85f74ece1f748843b09781fce0c7be9bd.json', 'cb24e4462716b5703697753dfdc9bfcaecb6452d.json', '0a15308c6bdc2ab87432c5d9aa541675dcab8c04.json', '1e2eff380a2a67997c1c2cfb38c7a4eddc396f09.json']


In [25]:
import json

# Load one JSON fulltext file
file_path = "/content/cord19/2020-11-29/document_parses/pdf_json/407dfb371c1bb8508e94ada9bea7cf86e55399b3.json"
with open(file_path, 'r') as f:
    paper = json.load(f)

# Extract full body text
body_text = paper.get("body_text", [])
full_text = " ".join([section["text"] for section in body_text])

print(full_text[:1000])  # print the first 1000 characters

"B lack swan" events are by definition unexpected (even if the "black swan" has been visible before) and come with catastrophic consequences. One such occurrence is the current coronavirus pandemic, which is likely to have a considerable impact on the practice of medicine in as yet unknown ways. A likely consequence will be a significant economic downturn and constrained budgets, resulting in even higher pressure to optimize our diagnostic and therapeutic pathways. To survive the cost pressure, diagnostic studies will have to evolve significantly: they must be fast, use the most cost effective approach, and result in a therapeutic consequences or management guidance. Shorter examinations require focus on the core question, standardized image acquisition and procedures, and rapid, increasingly automized data analysis and reporting. Cost-effectiveness requires more research, but also using the knowledge we have to reduce unnecessary diagnostic and therapeutic procedures. In this issue of

In [26]:
import os
import json

# Folder where your fulltext .json files are
fulltext_dir = "/content/cord19/2020-11-29/document_parses/pdf_json"
files = [f for f in os.listdir(fulltext_dir) if f.endswith(".json")]

# Let's collect chunks from the first N papers (you can increase later)
chunk_list = []
max_files = 100  # Start small for now

for fname in files[:max_files]:
    fpath = os.path.join(fulltext_dir, fname)
    with open(fpath, 'r') as f:
        paper = json.load(f)
        paper_id = paper.get("paper_id", "")
        title = paper.get("metadata", {}).get("title", "")

        for section in paper.get("body_text", []):
            text = section.get("text", "").strip()
            if len(text) > 100:  # Avoid very short lines
                chunk_list.append({
                    "paper_id": paper_id,
                    "title": title,
                    "text": text
                })

print(f"Extracted {len(chunk_list)} chunks.")

Extracted 3766 chunks.


In [27]:
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer
import numpy as np

# Load a lightweight embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Get just the text chunks
texts = [chunk["text"] for chunk in chunk_list]

# Generate embeddings (batch processing is handled by the model)
embeddings = model.encode(texts, show_progress_bar=True)

# Convert to a numpy array for easier handling later
embeddings = np.array(embeddings)
print("Embeddings shape:", embeddings.shape)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.7 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/118 [00:00<?, ?it/s]

Embeddings shape: (3766, 384)


In [28]:
import pickle

# Save both chunks and embeddings
with open("cord19_chunks.pkl", "wb") as f:
    pickle.dump(chunk_list, f)

with open("cord19_embeddings.npy", "wb") as f:
    np.save(f, embeddings)

In [29]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 71.7 MB/s eta 0:00:00


In [30]:
import faiss

# Dimensionality of embeddings
embedding_dim = embeddings.shape[1]

# Create FAISS index (Flat L2 distance)
index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to the index
index.add(embeddings)

# Check how many vectors are indexed
print(f"Number of vectors in index: {index.ntotal}")

Number of vectors in index: 3766


In [31]:
def search(query, top_k=5):
    # Encode the query
    query_embedding = model.encode([query])

    # Search the index
    distances, indices = index.search(query_embedding, top_k)

    # Print results
    print("\n🔎 Search Results:\n")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. {chunk_list[idx]['text'][:300]}...\n")

In [32]:
search("What are the symptoms of COVID-19?")


🔎 Search Results:

1. The most common symptoms of COVID-19 in children were a fever and cough. Other symptoms included fatigue, myalgia, nausea, vomiting, and diarrhea, which seems to be milder than adults with COVID-19 (Table 1) (20, 25, 81, 82) . Within 2,135 pediatric patients <18-years old who reported with COVID-19,...

2. We describe a cohort of 90 COVID-19 patients who were hospitalized in the Central Hospital of Wuhan. According to the COVID-19 Guidelines (seventh version), 20 out of 90 patients were included in the severe group, accounting for approximately 22.2% of the total cases, similar to the situations in pr...

3. Previous studies on the clinical characteristics of COVID-19 have shown that patients with COVID-19 may have mild or severe symptoms of acute respiratory infection; patients with mild disease have symptoms such as fever, dry cough, fatigue and abnormal chest CT, but their prognosis is good. In contr...

4. Among the 912 cases with COVID-19 we retrieved, 90 pa

#PUBMED

In [96]:
!pip install datasets --upgrade -q

In [99]:
from datasets import load_dataset

# Stream the dataset instead of downloading fully
dataset = load_dataset("pmc/open_access", split="train", streaming=True)

# Preview a few entries (e.g., first 5)
from itertools import islice

for example in islice(dataset, 5):
    print(example)
    print("-" * 80)

{'text': "==== Front\nPLoS BiolPLoS BiolpbioplosbiolPLoS Biology1544-91731545-7885Public Library of Science San Francisco, USA 10.1371/journal.pbio.0000005Research ArticleGenetics/Genomics/Gene TherapyInfectious DiseasesMicrobiologyPlasmodiumThe Transcriptome of the Intraerythrocytic Developmental Cycle of Plasmodium falciparum\n P. falciparum IDC TranscriptomeBozdech Zbynek \n1\nLlinás Manuel \n1\nPulliam Brian Lee \n1\nWong Edith D \n1\nZhu Jingchun \n2\nDeRisi Joseph L joe@derisilab.ucsf.edu\n1\n1Department of Biochemistry and Biophysics, University of California, San FranciscoSan Francisco, CaliforniaUnited States of America2Department of Biological and Medical Informatics, University of California, San FranciscoSan Francisco, CaliforniaUnited States of America10 2003 18 8 2003 18 8 2003 1 1 e512 6 2003 25 7 2003 Copyright: ©2003 Bozdech et al.2003This is an open-access article distributed under the terms of the Creative Commons Attribution License, which permits unrestricted use, 

In [100]:
def chunk_text(text, chunk_size=512, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunks.append(" ".join(words[i:i + chunk_size]))
    return chunks

In [101]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

for article in islice(dataset, 5):
    chunks = chunk_text(article["text"])
    embeddings = model.encode(chunks)
    print(f"{len(embeddings)} chunks embedded for article {article['accession_id']}")

29 chunks embedded for article PMC176545
11 chunks embedded for article PMC176546
1 chunks embedded for article PMC176547
2 chunks embedded for article PMC176548
15 chunks embedded for article PMC193604


In [102]:
!pip install faiss-cpu sentence-transformers datasets -q


In [103]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from itertools import islice

In [104]:
dataset = load_dataset("pmc/open_access", split="train", streaming=True)

In [105]:
def chunk_text(text, chunk_size=512, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunks.append(" ".join(words[i:i + chunk_size]))
    return chunks


In [106]:
model = SentenceTransformer("all-MiniLM-L6-v2")


In [107]:
# Initialize FAISS index
dim = 384  # for all-MiniLM-L6-v2
index = faiss.IndexFlatL2(dim)

# Store metadata (e.g. article titles, IDs)
metadata = []

# Process a small number first for testing
for i, article in enumerate(islice(dataset, 10)):
    try:
        text = article["text"]
        title = article.get("title", "No Title")
        chunks = chunk_text(text)

        embeddings = model.encode(chunks)
        index.add(np.array(embeddings).astype("float32"))

        # Store metadata for each chunk
        for chunk in chunks:
            metadata.append({
                "title": title,
                "chunk": chunk
            })

        print(f"✅ Article {i+1} processed. Total vectors: {index.ntotal}")
    except Exception as e:
        print(f"❌ Error on article {i+1}: {e}")


✅ Article 1 processed. Total vectors: 29
✅ Article 2 processed. Total vectors: 40
✅ Article 3 processed. Total vectors: 41
✅ Article 4 processed. Total vectors: 43
✅ Article 5 processed. Total vectors: 58
✅ Article 6 processed. Total vectors: 75
✅ Article 7 processed. Total vectors: 77
✅ Article 8 processed. Total vectors: 79
✅ Article 9 processed. Total vectors: 93
✅ Article 10 processed. Total vectors: 116


In [113]:
query = "what causes heart attacks"
query_embedding = model.encode([query]).astype("float32")

D, I = index.search(query_embedding, k=5)  # k = top 5 results

for idx in I[0]:
   # print(f"Title: {metadata[idx]['title']}")
    print(f"Chunk: {metadata[idx]['chunk'][:300]}...")
    print("-" * 80)

Chunk: the case that P. falciparum gene regulation is streamlined to the extent that it has lost the degree of dynamic flexibility observed in other unicellular organisms, from Escherichia coli to yeast. This observation also implies that disruption of a key transcriptional regulator, as opposed to a metab...
--------------------------------------------------------------------------------
Chunk: only a small fraction of it is bound to the Golgi membrane at any time [36]. It is not surprising, therefore, that some ARF1 may bind to the plasma membrane after being locally activated by ARNO, which is in turn recruited to the cell surface by the action of insulin. It should be remembered that ou...
--------------------------------------------------------------------------------
Chunk: J Biol Chem 1996 271 2422 2426 8576201 23. Shome K Vasudevan C Romero G ARF proteins mediate insulin-dependent activation of phospholipase D Curr Biol 1997 7 387 396 9197239 24. Shome K Nie Y Romero G ADP-ribo

In [116]:
import pickle

# Save FAISS index
faiss.write_index(index, "pubmed_faiss_index.index")

# Save metadata
with open("pubmed_metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)

In [117]:
# Load index
index = faiss.read_index("pubmed_faiss_index.index")

# Load metadata
with open("pubmed_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

# Example query
query = "COVID-19 vaccine development"
query_vector = model.encode([query]).astype("float32")
D, I = index.search(query_vector, k=5)

# Show results
for idx in I[0]:
    print("Title:", metadata[idx]["title"])
    print("Snippet:", metadata[idx]["chunk"][:300], "\n---\n")

Title: No Title
Snippet: genes in merozoites (Kaneko et al. 2001). Figure 5 Phaseogram of Putative Vaccine Targets The similarity of all expression profiles to seven known vaccine candidates (boxed) was calculated. The top 5% of similar profiles correspond to 262 ORFs, 28 of which have been previously associated with plasmo 
---

Title: No Title
Snippet: the total intensity was determined using the signal in the 3D7 channel as the total amount of intensity for each oligonucleotide; intensity differences less than 50% were considered to be significant for subsequence analysis. Calculation for in-phase plastid-targeted genes. The range of FFT-based ph 
---

Title: No Title
Snippet: putative proteases throughout the entire IDC (Table S6). This set includes over 65 proteases from a group of recently predicted proteases (Wu et al. 2003). The differing temporal expression of these proteases may allow for a multifaceted approach toward identifying protease inhibitors with efficacy  
---

Titl

In [119]:
import pickle

# Load the pickle file
with open("/content/cord19_chunks.pkl", "rb") as f:
    chunks = pickle.load(f)

# Check how it looks
print(f"Total chunks: {len(chunks)}")
print("Sample chunk:\n", chunks[0])

Total chunks: 3766
Sample chunk:
 {'paper_id': '407dfb371c1bb8508e94ada9bea7cf86e55399b3', 'title': 'Stress-Only CMR', 'text': '"B lack swan" events are by definition unexpected (even if the "black swan" has been visible before) and come with catastrophic consequences. One such occurrence is the current coronavirus pandemic, which is likely to have a considerable impact on the practice of medicine in as yet unknown ways. A likely consequence will be a significant economic downturn and constrained budgets, resulting in even higher pressure to optimize our diagnostic and therapeutic pathways.'}


In [124]:
!pip install sacremoses -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 48.6 MB/s eta 0:00:00


In [121]:
!pip install transformers datasets accelerate -q

In [125]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load tokenizer and model
model_name = "microsoft/BioGPT-Large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.29G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

BioGptForCausalLM(
  (biogpt): BioGptModel(
    (embed_tokens): BioGptScaledWordEmbedding(57717, 1600, padding_idx=1)
    (embed_positions): BioGptLearnedPositionalEmbedding(2050, 1600)
    (layers): ModuleList(
      (0-47): 48 x BioGptDecoderLayer(
        (self_attn): BioGptSdpaAttention(
          (k_proj): Linear(in_features=1600, out_features=1600, bias=True)
          (v_proj): Linear(in_features=1600, out_features=1600, bias=True)
          (q_proj): Linear(in_features=1600, out_features=1600, bias=True)
          (out_proj): Linear(in_features=1600, out_features=1600, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1600, out_features=6400, bias=True)
        (fc2): Linear(in_features=6400, out_features=1600, bias=True)
        (final_layer_norm): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm): LayerNorm((

In [130]:
from datasets import load_dataset

dataset = load_dataset("GBaker/MedQA-USMLE-4-options")

In [135]:
print(dataset["train"][0])

{'question': 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?', 'answer': 'Nitrofurantoin', 'options': {'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': 'Doxycycline', 'D': 'Nitrofurantoin'}, 'meta_info': 'step2&3', 'answer_idx': 'D', 'metamap_phrases': ['23 year old pregnant woman', 'weeks presents', 'burning', 'urination', 'states', 'started 1 day', 'worsening', 'drinking', 'water', 'taking cranberry extract', 'feels well', 'followed by', 'doctor', 'pregnancy', '

In [136]:
def format_medqa(example):
    prompt = (
        f"Question: {example['question']}\n"
        f"A. {example['choice_A']}\n"
        f"B. {example['choice_B']}\n"
        f"C. {example['choice_C']}\n"
        f"D. {example['choice_D']}\n"
        f"Answer:"
    )
    completion = f" {example['answer']}"
    return {"prompt": prompt, "completion": completion}

In [139]:
# Check the first few entries of the dataset
dataset["train"][:5]  # Print first 5 examples from the training set

{'question': ['A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?',
  'A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?',
  "A mother brings her 3-week-old infant to the pediatrician's office because she is concerned about his feeding habits. He was bor

In [146]:
def format_medqa(example):
    prompt = (
        f"Question: {example['question']}\n"
        f"A. {example['options']['A']}\n"
        f"B. {example['options']['B']}\n"
        f"C. {example['options']['C']}\n"
        f"D. {example['options']['D']}\n"
        f"Answer:"
    )
    completion = f" {example['answer']}"
    return {"prompt": prompt, "completion": completion}

In [148]:
# Format the dataset
formatted_dataset = dataset.map(format_medqa)

# Tokenize the formatted dataset
def tokenize_function(examples):
    return tokenizer(examples["prompt"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/10178 [00:00<?, ? examples/s]

Map:   0%|          | 0/1273 [00:00<?, ? examples/s]

Map:   0%|          | 0/10178 [00:00<?, ? examples/s]

Map:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [151]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=None)

In [153]:
# Check available splits in the dataset
print(tokenized_dataset)


DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases', 'prompt', 'completion', 'input_ids', 'attention_mask'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases', 'prompt', 'completion', 'input_ids', 'attention_mask'],
        num_rows: 1273
    })
})


In [154]:
# Using the test split as validation
train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["test"]

In [155]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the model and tokenizer
model_name = "t5-small"  # You can choose other models like "t5-base" or "t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [156]:
from transformers import DataCollatorForSeq2Seq

# Define the data collator for T5 model
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [170]:
from transformers import Trainer, TrainingArguments

# Define training arguments
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Output directory for model checkpoints
    eval_strategy="epoch",           # Evaluate at the end of each epoch (updated)
    save_strategy="epoch",           # Save model at the end of each epoch (updated)
    learning_rate=2e-5,              # Learning rate for training
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    num_train_epochs=3,              # Number of epochs
    weight_decay=0.01,               # Weight decay for regularization
    logging_dir="./logs",            # Directory to store logs
    logging_steps=10,                # Log every 10 steps
    load_best_model_at_end=True,     # Load the best model at the end of training
    metric_for_best_model="accuracy",  # Metric to evaluate the best model
)


# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


<ipython-input-170-737e5d5ee299>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [165]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [166]:
trainer.train()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [169]:
# Check tokenized inputs
print("Sample tokenized input:", tokenized_dataset["train"][0])

# Check if there are any invalid labels (out of range)
num_labels = 2  # Update this based on your task (e.g., binary classification -> num_labels=2)
labels = tokenized_dataset["train"]["labels"]

# Ensure labels are within the correct range
assert min(labels) >= 0, "Invalid label detected!"
assert max(labels) < num_labels, "Invalid label detected!"

print("Dataset seems correct!")


Sample tokenized input: {'question': 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?', 'answer': 'Nitrofurantoin', 'options': {'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': 'Doxycycline', 'D': 'Nitrofurantoin'}, 'meta_info': 'step2&3', 'answer_idx': 'D', 'metamap_phrases': ['23 year old pregnant woman', 'weeks presents', 'burning', 'urination', 'states', 'started 1 day', 'worsening', 'drinking', 'water', 'taking cranberry extract', 'feels well', 'followed by', 

KeyError: "Column labels not in the dataset. Current columns in the dataset: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases', 'prompt', 'completion', 'input_ids', 'attention_mask']"

In [168]:
import torch
torch.cuda.empty_cache()


In [172]:
# Set CUDA environment variable for better error reporting
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Install required packages
!pip install transformers datasets accelerate -q

import torch
from datasets import load_dataset
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)

# Load the dataset
print("Loading dataset...")
dataset = load_dataset("GBaker/MedQA-USMLE-4-options")
print("Dataset structure:", dataset)
print("Sample entry:", dataset["train"][0])

# Check if GPU is available and working properly
print("Checking GPU availability...")
if torch.cuda.is_available():
    try:
        # Test GPU with a small tensor operation first
        test_tensor = torch.tensor([1.0, 2.0, 3.0], device="cuda")
        print("GPU test successful:", test_tensor.device)
        device = torch.device("cuda")
    except RuntimeError as e:
        print(f"GPU error: {e}")
        print("Falling back to CPU")
        device = torch.device("cpu")
else:
    print("No GPU available, using CPU")
    device = torch.device("cpu")

print(f"Using device: {device}")

# Load T5 model and tokenizer
print("Loading model and tokenizer...")
model_name = "t5-small"  # Using smaller model to avoid memory issues
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Load model with proper error handling
try:
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    model.to(device)
    print("Model loaded and moved to device successfully")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Trying with CPU only...")
    device = torch.device("cpu")
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    model.to(device)

# Format the dataset for T5
print("Formatting dataset...")
def format_medqa(example):
    # For T5, we need to format input as a question
    # Notice that we're using answer_idx and converting it to the actual answer
    input_text = f"answer multiple choice medical question: {example['question']} Options: "
    for key, value in example['options'].items():
        input_text += f"{key}: {value}. "

    # The target is the letter answer (A, B, C, D)
    target_text = example['answer_idx']  # Using the letter answer directly

    return {
        "input_text": input_text,
        "target_text": target_text
    }

# Apply formatting to dataset
formatted_dataset = dataset.map(format_medqa)

# Print a sample formatted entry
print("\nSample formatted entry:")
print(formatted_dataset["train"][0])

# Tokenize the dataset properly for T5
print("Tokenizing dataset...")
def tokenize_function(examples):
    inputs = tokenizer(
        examples["input_text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    # Tokenize the targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target_text"],
            padding="max_length",
            truncation=True,
            max_length=8,
            return_tensors="pt"
        )

    # Replace padding token id's with -100 so they're not included in loss computation
    labels_with_ignore = labels["input_ids"].clone()
    labels_with_ignore[labels_with_ignore == tokenizer.pad_token_id] = -100

    inputs["labels"] = labels_with_ignore
    return inputs

# Apply tokenization with batched=False to avoid memory issues
tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=8,  # Smaller batch size
    remove_columns=["input_text", "target_text", "question", "options", "answer", "meta_info", "answer_idx", "metamap_phrases"]
)

print("\nTokenized dataset structure:", tokenized_dataset)

# Create data collator with padding
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True
)

# Define training arguments with smaller batch sizes
print("Setting up training configuration...")
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,   # Smaller batch size to avoid OOM
    per_device_eval_batch_size=4,
    num_train_epochs=2,              # Reduced epochs for testing
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    report_to="none",                # Disable wandb reporting
    # Add fp16 for faster training if on compatible GPU
    fp16=torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 7,
)

# Initialize the Trainer
print("Initializing trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Free up memory before training
if device.type == "cuda":
    torch.cuda.empty_cache()

# Start training with error handling
print("Starting training...")
try:
    trainer.train()
    print("Training completed successfully!")

    # Save the fine-tuned model
    model.save_pretrained("./medqa-t5-model")
    tokenizer.save_pretrained("./medqa-t5-model")

    # Test the model on a sample question
    def generate_answer(question, options):
        input_text = f"answer multiple choice medical question: {question} Options: "
        for key, value in options.items():
            input_text += f"{key}: {value}. "

        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_length=8)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return answer

    # Try it on a sample from the test set
    sample = dataset["test"][0]
    print("\nSample question:")
    print(sample["question"])
    print("Options:", sample["options"])
    print("Correct answer:", sample["answer_idx"])

    predicted = generate_answer(sample["question"], sample["options"])
    print("Model prediction:", predicted)

except Exception as e:
    print(f"Training error: {e}")
    print("Troubleshooting recommendations:")
    print("1. Try using a smaller model like t5-small")
    print("2. Further reduce batch size")
    print("3. Consider training on CPU if GPU memory is limited")
    print("4. Check for any dataset inconsistencies")

Loading dataset...
Dataset structure: DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 1273
    })
})
Sample entry: {'question': 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?', 'answer': 'Nitrofurantoin', 'op

Map:   0%|          | 0/10178 [00:00<?, ? examples/s]

Map:   0%|          | 0/1273 [00:00<?, ? examples/s]


Sample formatted entry:
{'question': 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?', 'answer': 'Nitrofurantoin', 'options': {'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': 'Doxycycline', 'D': 'Nitrofurantoin'}, 'meta_info': 'step2&3', 'answer_idx': 'D', 'metamap_phrases': ['23 year old pregnant woman', 'weeks presents', 'burning', 'urination', 'states', 'started 1 day', 'worsening', 'drinking', 'water', 'taking cranberry extract', 'feels well', 'followed by',

Map:   0%|          | 0/10178 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1273 [00:00<?, ? examples/s]


Tokenized dataset structure: DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1273
    })
})
Setting up training configuration...
Initializing trainer...


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-172-d3d07584a0f0>:150: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
